### Banking (C)

Consider following C program with a global variable, `balance`. Functions `deposit` and `withdraw` are for creating threads that will deposit to and withdraw by incrementing and decrementing `balance`. They do so by depositing or withdrawing successively increasing amounts from `1` to `M`. The main program creates `N` threads, half `deposit` and half `withdraw` threads.

In [8]:
%%writefile Bank.c
#include <pthread.h>
#include <stdio.h>

#define M 10000 /* maximum amount deposited or withdrawn */
#define N 50 /* total number of deposit, withdraw threads */

int balance = 0;

void* deposit(void *arg) {
    for (int i = 1; i <= M; ++i)
        balance += i;
}

void* withdraw(void *arg) {
    for (int i = 1; i <= M; ++i)
        balance -= i;
}

int main(void) {
    pthread_t w[N];
    for (int i = 0; i < N; i++)
        if (i % 2) pthread_create(&w[i], NULL, withdraw, NULL);
        else pthread_create(&w[i], NULL, deposit, NULL);
    for (int i = 0; i < N; i++)
        pthread_join(w[i], NULL); 
    printf("%d\n", balance);
}

Overwriting Bank.c


In [9]:
!gcc Bank.c -lpthread -o Bank

In [10]:
%time !./Bank

-1473570496
CPU times: user 11.4 ms, sys: 9.38 ms, total: 20.8 ms
Wall time: 859 ms


Since in total the same amount is deposited as withdrawn, the balance at the end should be zero. Run it with varying values of `M` and `N`. Sometimes the final balance will be zero and sometimes not. Give the values for `M` and `N` that show different behavious and explain it!  [2 points]

M = 1000, N = 1000
M = 10000, N = 10000
M = 10000, N = 50

C has built-in functions `__atomic_add_fetch` and `__atomic_sub_fetch` to atomically increment and decrement an integer variable, see https://gcc.gnu.org/onlinedocs/gcc/_005f_005fatomic-Builtins.html. Copy above code to the cell bellow and modify it to use those functions! For the last parameter, `memorder`, you can use `__ATOMIC_SEQ_CST`. Test if you now get zero as the final balance! [2 points]

In [1]:
%%writefile Bank.c
#include <pthread.h>
#include <stdio.h>

#define M 1000 /* maximum amount deposited or withdrawn */
#define N 1000 /* total number of deposit, withdraw threads */

int balance = 0;

void* deposit(void *arg) {
    for (int i = 1; i <= M; ++i)
        __atomic_add_fetch(&balance, i, __ATOMIC_SEQ_CST);
}

void* withdraw(void *arg) {
    for (int i = 1; i <= M; ++i)
        __atomic_sub_fetch(&balance, i, __ATOMIC_SEQ_CST);
}

int main(void) {
    pthread_t w[N];
    for (int i = 0; i < N; i++)
        if (i % 2) pthread_create(&w[i], NULL, withdraw, NULL);
        else pthread_create(&w[i], NULL, deposit, NULL);
    for (int i = 0; i < N; i++)
        pthread_join(w[i], NULL); 
    printf("%d\n", balance);
}

Overwriting Bank.c


In [2]:
!gcc Bank.c -lpthread -o Bank

In [3]:
%time !./Bank

0
CPU times: user 2.01 ms, sys: 8.21 ms, total: 10.2 ms
Wall time: 173 ms


C also allow variables to be declared as `_Atomic`, e.g. `_Atomic int global`, which makes updates of those variables implicitly atomic. If `stdatomic.h` in included, then `atomic_int global` is an equivalent to the above declaration. Copy the code from the first code cell to the cell bellow and modify it accordingly! Test if you now get zero as the final balance! [2 points]

In [42]:
%%writefile Bank.c
#include <pthread.h>
#include <stdio.h>
#include <stdatomic.h>

#define M 10000 /* maximum amount deposited or withdrawn */
#define N 100000 /* total number of deposit, withdraw threads */

_Atomic int balance = 0;

void* deposit(void *arg) {
    for (int i = 1; i <= M; ++i)
        balance += i;
}

void* withdraw(void *arg) {
    for (int i = 1; i <= M; ++i)
        balance -= i;
}

int main(void) {
    pthread_t w[N];
    for (int i = 0; i < N; i++)
        if (i % 2) pthread_create(&w[i], NULL, withdraw, NULL);
        else pthread_create(&w[i], NULL, deposit, NULL);
    for (int i = 0; i < N; i++)
        pthread_join(w[i], NULL); 
    printf("%d\n", balance);
}

Overwriting Bank.c


In [43]:
!gcc Bank.c -lpthread -Wno-incompatible-pointer-types -o Bank

In [44]:
%time !./Bank

CPU times: user 78.5 ms, sys: 17.7 ms, total: 96.2 ms
Wall time: 6.11 s


The `%time` [line magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) displays the *wall clock time* of the subsequent command (ignore the other displayed times). Choose values of `M` and `N` that make the wall clock time in the seconds. Run each timing command repeatedly to see if you get similar results; choose a time of the day when there is little load on the server. Now compare the times of the three implementations for the same value of `M` and `N`. What is your observation and what is your explanation?

The first implementation did not use synchronization mechanisms which led to race conditions that provided non zero outputs. The second implementation used atomic increment and decrement which provided the correct output of 0 continuously. In the final implementation we used an atomic variable which also gave consistent answers of 0. the wall time for each execution increased depending on M and N but the overall execution of the code was faster.